In [7]:
import numpy as np
import pandas as pd
import xarray as xr
import math
from sklearn.neighbors import BallTree as BallTree
import cartopy.crs
import cartopy.feature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.colors import LinearSegmentedColormap
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [8]:
bath = xr.open_dataset(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_metOcean_bathy.nc')
depthNS = 20
shp_coast2 = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\Continental shapefile coordinates2.xlsx')
ERA5_on_coast = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_locs_on_coast_fin.xlsx')
ERA5_around_coast = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_coordinates_around_coast_fin.xlsx')
X1 = ERA5_on_coast.iloc[:,0]; Y1 = ERA5_on_coast.iloc[:,1]
X2 = ERA5_around_coast.iloc[:,0]; Y2 = ERA5_around_coast.iloc[:,1]
lfhs20 = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\lf_waveheight_lookuptable_d20.xlsx',header=None)
lftm20 = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\lf_waveperiod_lookuptable_d20.xlsx',header=None)
pi = math.pi
grav = 9.81
seasons = np.array(['DJF', 'MAM', 'JJA', 'SON'])

In [52]:
largeports = [1465,3671,3304,3701,1845]#,1644,2075,3183,3360,1469,1960,2087,2043,3562,2957,1130,1493,1613,3499,1588,1606,950,3272,1609,1559,2999,899,1419,2235,3096,3385,1386,1548,1473,2378,1623,2882,2724,204,1167,2885,1552,1551,1628,2165,3669,4523,4520,2681,1378,3705,3122,2600,3921,2061,1820,1861,1907,4255,1512,1821,2102,2124,2430,1940,4245,4068,1540,1563,3746,1880,1916,1711,4291,3797,3390,921,1647,1739,371,663,2905,1575,675,674,508,738,2167,1372,2146,461,616,657,2013,1198,1182,2218,1946,1950,1776,338,192,387]
locs = range(0,5)#103)
cols = ['lon', 'lat', 'depth0','shoreN','tm','tp','Ks','Kr','wind50DJF','wind50MAM','wind50JJA','wind50SON','wind95','windAv','swh50','swh95','swhAv','ig50','ig95','igAv','wavePo','windAvUp','swhAvUp','igAvUp','operability','frequency','duration']
df = pd.DataFrame(columns = cols, index=range(0,103))

In [53]:
# THRESHOLDS
Twind = 13.8
Twave = 2.0
Tig = 0.05

In [60]:
for i, p in zip(locs,largeports):
    print(p)
    # LOAD ERA5 DATA
    ds = xr.open_dataset(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_output_world_coast\Loc%s.nc' % p)
    dfx = ds.to_dataframe()
    dfx = dfx.dropna()
    if dfx.empty: continue
    
    dfx['season'] = seasons[(dfx.index.month // 3) % 4]
    df.loc[i].lon = dfx.longitude[0]
    df.loc[i].lat = dfx.latitude[0]
    
    # DEPTH OFFSHORE
    depth = bath.sel(latitude=dfx.latitude[0], longitude=dfx.longitude[0], method='nearest')
    depth = depth.to_dataframe()
    df['depth0'].loc[i] = depth.wmb[0]
    if np.isnan(df.loc[i].depth0) or df.loc[i].depth0 <= depthNS: df.loc[i].depth0 = depthNS
     
    # SHORENORMAL ORIENTATION
    F = np.array([df.loc[i].lon,df.loc[i].lat]).reshape(1,-1)
    x1 = shp_coast2.iloc[:,0].tolist()
    y1 = shp_coast2.iloc[:,1].tolist()
    X = np.column_stack((x1,y1))
    tree = BallTree(X, leaf_size=100)             
    dist, ind = tree.query(F, k=1)
    Cordmin = (X[ind-4]+X[ind-3]+X[ind-2]+X[ind-1])/4
    Cordpls = (X[ind+4]+X[ind+3]+X[ind+2]+X[ind+1])/4
    coastcoord = (Cordmin+Cordpls)/2
    theta = math.atan2((df.loc[i].lon - coastcoord[0,0,0]),(df.loc[i].lat - coastcoord[0,0,1]))
    shoreN = (theta*180/(math.pi)) % 360
    if np.isnan(shoreN):
        theta = math.atan2(X2[i]-X1[i],Y2[i]-Y1[i])
        shoreN = (theta*180/(math.pi)) % 360
    df['shoreN'].loc[i] = shoreN
    
    # WIND
    for s in seasons:
        print(j)
        dfs = dfx.query('season == "%s"' % j)
        wind = np.sqrt(dfs.u10**2 + dfs.v10**2); wind[-1] = 0; wind[0] = 0
        dfs['wind'] = wind
        df['wind50%s' % j].loc[i] = wind.quantile([0.50]).iloc[0]
        df['wind95'].loc[i] = wind.quantile([0.95]).iloc[0]
        df['windAv'].loc[i] = np.count_nonzero(wind < Twind)/len(wind)*100
        idx = np.argwhere(np.diff(np.sign(Twind - wind))).flatten()
        Events = idx.reshape(len(idx)//2,2)
        if idx.size != 0:
            IntervalIndex = np.column_stack((Events[:-1,1],Events[1:,0]))
            ShortIntervalIndex = IntervalIndex[Events[1:,0]-Events[:-1,1] <= 4]
            for n in range(0,len(ShortIntervalIndex)):
                x = np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)
                wind[x] = Twind
            dfs.wind = wind
            df.loc[i].windAvUp = np.count_nonzero(wind < Twind)/len(wind)*100
        else:
            df.loc[i].windAvUp = 100

        # NEARSHORE TRANSLATION
        dirWave = dfs.mwd
        waveHeight = dfs.swh    
        wavePeriod = dfs.mwp
        peakPeriod = dfs.pp1d
        dirWave = dirWave * 1.
        waveHeight = waveHeight.astype('float')
        wavePeriod = wavePeriod * 1.
        depthOffshore = float(df.loc[i].depth0)
        depthNearshore = float(depthNS)
        shoreNormal = float(shoreN % 360)
        df.loc[i].tp = np.mean(peakPeriod)

        # WAVE LENGHT/NUMBER (DEEP & NEAR) BASED ON THE WAVE PERIOD WHICH REMAINS CONSTANT
        # Calculation based on the Matlab function disper.m and transformed to nearshore

        w2 = (2 * pi / wavePeriod)**2 * depthOffshore / grav
        q = w2 / ( (1 - np.exp(-(w2**(5/4))))**(2/5))
        for j in range(2):
            thq = np.tanh(q)
            thq2 = 1 - thq**2
            a = (1 - q * thq)*thq2
            b = thq + q * thq2
            c = q * thq - w2
            arg = b**2 - (4*a*c)
            arg = (-b + np.sqrt(arg)) / (2 * a)
            iq = np.abs(a * c) < (10**-8 * b**2)
            arg[iq] = - c[iq] / b[iq]
            q = q + arg
        k = q / depthOffshore
        Ldeep = 2 * pi / k

        # Nearshore depth wave number and wave length calculation via dispersion relation
        w3 = (2 * pi / wavePeriod)**2 * depthNearshore / grav
        q1 = w3 / (1 - np.exp(-(w3**(5/4))))**(2/5)
        for m in range(2):
            thq = np.tanh(q1)
            thq2 = 1 - thq**2
            a = (1 - q1 * thq)*thq2
            b = thq + q1 *thq2
            c = q1 * thq - w3
            arg = b**2 - (4*a*c)
            arg = (-b + np.sqrt(arg)) / (2 * a)
            iq = np.abs(a * c) < (10**-8 * b**2)
            arg[iq] = - c[iq] / b[iq]
            q1 = q1 + arg
        k1 = q1 / depthNearshore
        Lnear = 2 * pi / k1

        # Calculate the waveVelocity (deep and near)
        Cdeep = ((grav/k) * np.tanh(k * depthOffshore))**0.5
        Cnear = ((grav/k1) * np.tanh(k1 * depthNearshore))**0.5

        # Calculate the wave-angle of the point offshore
        relDir = (dirWave - shoreNormal + 180.) % 360. - 180.
        SinThetadeep = np.sin(relDir * (pi/180.))

        # Calculate the wave-angle of the point near the coast
        SinThetanear = (SinThetadeep / Cdeep) * Cnear
        relDirnear = np.arcsin(SinThetanear) * (180./pi)

        # Refraction coefficient
        Kr = (np.cos(relDir * (pi/180.)) / np.cos(relDirnear * (pi/180.)))**0.5
        Kr[(Kr < 0.8)] = 0.8
        df.loc[i].Kr = np.mean(Kr)

        # Calculate group wave velocities
        waveNumberdeep = 2*pi / Ldeep
        if depthOffshore/Ldeep.any() > 0.5: ndeep = 0.5
        else: ndeep = 0.5 * (1 + (2 * waveNumberdeep * depthOffshore) / np.sinh(2 * waveNumberdeep * depthOffshore))
        waveVelocitydeep = ( (grav*Ldeep / 2*pi) * np.tanh(2*pi * depthOffshore / Ldeep) )**0.5
        Cgroupdeep = ndeep * waveVelocitydeep
        waveNumbernear = 2*pi / Lnear
        nnear = 0.5 * (1 + (2 * waveNumbernear * depthNearshore) / np.sinh(2 * waveNumbernear * depthNearshore))
        waveVelocitynear = ( (grav*Lnear / 2*pi) * np.tanh(2*pi * depthNearshore / Lnear) )**0.5
        Cgroupnear = nnear * waveVelocitynear

        # Shoaling coefficient:
        Ks = (Cgroupdeep / Cgroupnear) ** 0.5
        df.loc[i].Ks = np.mean(Ks)

        # Wave-climate of the near shore
        waveHeightnearAlldir = Ks * waveHeight
        waveHeightnearAlldir[waveHeightnearAlldir > (depthNearshore*0.73)] = depthNearshore*0.73
        dfs['swh_nr_ad'] = waveHeightnearAlldir
        waveHeightnearReldir = Kr * Ks * waveHeight
        waveHeightnearReldir[waveHeightnearReldir > (depthNearshore*0.73)] = depthNearshore*0.73
        dfs['swh_nr_rd'] = waveHeightnearReldir
        wavePeriodnear = wavePeriod * 1.
        df.loc[i].tm = np.mean(wavePeriodnear)
        ixD = ~((-90 < relDir) & (relDir < 90))
        waveHeightnearReldir[ixD] = float('NaN')
        waveHeightnearReldir[ixD] = float('NaN')
        relDirnear[ixD] = relDir[ixD]
        df['swh50'].loc[i] = waveHeightnearReldir.quantile([0.5]).iloc[0]
        df['swh95'].loc[i] = waveHeightnearReldir.quantile([0.95]).iloc[0]
        waveHeightnearReldir = waveHeightnearReldir.fillna(0)
        df['swhAv'].loc[i] = np.count_nonzero(waveHeightnearReldir < Twave)/len(waveHeightnearReldir)*100
        waveHeightnearReldir[-1] = 0; waveHeightnearReldir[0] = 0
        idx = np.argwhere(np.diff(np.sign(Twave - waveHeightnearReldir))).flatten()
        Events = idx.reshape(len(idx)//2,2)
        if idx.size != 0:
            IntervalIndex = np.column_stack((Events[:-1,1],Events[1:,0]))
            ShortIntervalIndex = IntervalIndex[Events[1:,0]-Events[:-1,1] <= 4]
            for n in range(0,len(ShortIntervalIndex)):
                x = np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)
                waveHeightnearReldir[x] = Twave
            dfs.swh_nr_rd = waveHeightnearReldir
            df.loc[i].swhAvUp = np.count_nonzero(waveHeightnearReldir < Twave)/len(waveHeightnearReldir)*100
        else:
            df.loc[i].swhAvUp = 100

        # Infragravity (bound) waves nearshore
        idhs = (dfs.swh_nr_ad * 20).round().astype(int)
        idhs[idhs > 249]=249
        idtp = (peakPeriod * 10).round().astype(int)
        idtp[idtp > 249]=249
        lfhs = np.array(lfhs20)
        lftm = np.array(lftm20)
        lfhs = lfhs[idhs,idtp]
        lftm = lftm[idhs,idtp]
        lfhs[-1] = 0; lfhs[0] = 0
        dfx['lfhs'] = lfhs
        dfx['lftm'] = lftm
        df['ig50'].loc[i] = dfs.lfhs.quantile([0.50]).iloc[0]
        df['ig95'].loc[i] = dfs.lfhs.quantile([0.95]).iloc[0]
        df['igAv'].loc[i] = np.count_nonzero(lfhs < Tig)/len(lfhs)*100
        idx = np.argwhere(np.diff(np.sign(Tig - lfhs))).flatten()
        Events = idx.reshape(len(idx)//2,2)
        if idx.size != 0:
            IntervalIndex = np.column_stack((Events[:-1,1],Events[1:,0]))
            ShortIntervalIndex = IntervalIndex[Events[1:,0]-Events[:-1,1] <= 4]
            for n in range(0,len(ShortIntervalIndex)):
                lfhs[np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)] = Tig
            dfs.lfhs = lfhs
            df.loc[i].igAvUp = np.count_nonzero(lfhs < Tig)/len(lfhs)*100
        else:
            df.loc[i].igAvUp = 100

        # Longshore Sediment Transport potential
        dfs['wavePo'] = waveHeightnearAlldir**2.5*np.sin(2*SinThetanear)
        df['wavePo'].loc[i] = np.sum(np.abs(dfx.wavePo))/len(waveHeightnearAlldir)

        #Calculate the operability
        dfy = pd.concat([dfs.wind >= Twind, dfs.swh_nr_rd >= Twave, dfs.lfhs >= Tig], axis=1)
        dfz = dfy[dfy.wind | dfy.swh_nr_rd | dfy.lfhs]
        df['operability'].loc[i] = (len(dfy.index) - len(dfz.index))/len(dfy.index)*100

        #Calculate the frequency (per year) shut down and restart again limits the uptime
        freq = dfy.sum(axis=1)
        if np.sum(freq) != 0:
            idx2 = np.argwhere(np.diff(freq > 0)).flatten()
            df['frequency'].loc[i] = len(idx2.reshape(len(idx2)//2,2))/40

        #Calculate the duration (per event) short periods could be caught up more quickly
            Events2 = idx2.reshape(len(idx2)//2,2)
            df['duration'].loc[i] = np.sum(Events2[:,1]-Events2[:,0])/len(Events2)
        else: df['frequency'].loc[i] = 0; df['duration'].loc[i] = 0

1465
1


IndexError: index -1 is out of bounds for axis 0 with size 0

In [34]:
#dfx.sel(season=DJF)

#dfx.loc[dfx['season'] == DJF]
#To select rows whose column value is in an iterable, some_values, use isin:

dfx[dfx['season'].isin(DJF)]

NameError: name 'DJF' is not defined

In [6]:
#df.to_excel('output_103ports_d20_w138_s_2_i_005_seasons.xlsx')

In [59]:
dfx

,latitude,longitude,mwd,mwp,pp1d,swh,u10,v10,season,swh_nr_ad,swh_nr_rd,lfhs,lftm,wavePo
time,,,,,,,,,,,,,,
1979-01-01 00:00:00,55.0,-5.5,41.041748,4.351982,5.118543,0.745653,-2.557667,-4.218767,DJF,0.744521,0.000000,0.000000,12.231238,-0.147427
1979-01-01 01:00:00,55.0,-5.5,39.651901,4.310612,5.041320,0.647746,-1.733435,-4.044945,DJF,0.646892,0.646889,0.003017,35.126319,-0.118932
1979-01-01 02:00:00,55.0,-5.5,37.866516,4.273755,4.957865,0.566748,-0.603384,-3.764898,DJF,0.566092,0.566088,0.002160,35.126319,-0.098815
1979-01-01 03:00:00,55.0,-5.5,35.229645,4.235644,4.812423,0.500204,0.785057,-3.625841,DJF,0.499700,0.499696,0.001894,33.608768,-0.086439
1979-01-01 04:00:00,55.0,-5.5,28.774826,4.134099,4.704381,0.454387,2.173498,-3.637428,DJF,0.454077,0.454071,0.001583,32.814395,-0.092226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,55.0,-5.5,250.117889,4.184746,4.214382,0.936831,8.669243,3.338634,DJF,0.936051,0.000000,0.017307,8.375133,-0.536869
2018-12-31 20:00:00,55.0,-5.5,254.007263,4.259212,4.293336,0.973103,9.051923,2.707081,DJF,0.972034,0.000000,0.017514,8.214737,-0.675728
2018-12-31 21:00:00,55.0,-5.5,258.720673,4.322396,4.352204,1.011283,9.410072,1.668011,DJF,1.009895,0.000000,0.018502,8.474054,-0.838974


In [35]:
dfx.query('season == "MAM"')

,latitude,longitude,mwd,mwp,pp1d,swh,u10,v10,season,wind,swh_nr_ad,swh_nr_rd,lfhs,lftm,wavePo
time,,,,,,,,,,,,,,,
1979-03-01 00:00:00,40.5,14.0,293.065857,3.578737,4.480331,0.237845,-1.393274,0.634733,MAM,1.531045,0.237836,0.000000,0.000522,31.246747,0.025618
1979-03-01 01:00:00,40.5,14.0,290.055420,3.335030,4.399300,0.238664,-1.623863,1.227661,MAM,2.035702,0.238662,0.000000,0.000540,30.485690,0.025624
1979-03-01 02:00:00,40.5,14.0,280.046326,3.135450,4.337315,0.243300,-1.914961,1.859215,MAM,2.669036,0.243300,0.242582,0.000559,29.681325,0.026550
1979-03-01 03:00:00,40.5,14.0,245.618729,2.991783,4.287103,0.249573,-2.253487,2.199133,MAM,3.148712,0.249573,0.249573,0.000559,29.681325,0.031034
1979-03-01 04:00:00,40.5,14.0,200.297562,2.878203,2.414025,0.257754,-2.683592,2.425102,MAM,3.617013,0.257754,0.257754,0.004305,3.938907,0.011435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-31 19:00:00,40.5,14.0,272.833405,3.727419,3.652700,0.483841,2.955212,-2.293203,MAM,3.740596,0.483793,0.483535,0.007588,6.161050,0.149173
2018-05-31 20:00:00,40.5,14.0,272.344482,3.772800,3.733732,0.473205,3.575022,-1.286966,MAM,3.799614,0.473145,0.472851,0.005811,6.394600,0.141242
2018-05-31 21:00:00,40.5,14.0,272.685059,3.709116,3.801259,0.482750,4.495744,-0.950911,MAM,4.595209,0.482707,0.482486,0.007081,6.424537,0.148382


In [36]:
df

,lon,lat,depth0,shoreN,tm,tp,Ks,Kr,wind50,wind95,...,ig50,ig95,igAv,wavePo,windAvUp,swhAvUp,igAvUp,operability,frequency,duration
0,-5.5,55,63.442,50.0591,4.7986,6.33917,0.986807,0.973634,7.3775,13.6449,...,0.0109026,0.0413728,97.4336,1.15736,95.1283,99.0469,97.3922,94.7502,44.3,10.3883
1,129.5,35,118.259,7.86986,5.10168,6.18808,0.981588,0.992271,6.61044,12.2928,...,0.0101455,0.0448219,96.4302,1.43841,97.8208,94.4456,96.3775,93.4523,37.575,15.2754
2,120.5,35.5,36.792,146.08,4.39909,4.94891,0.993142,0.993889,5.44449,10.5677,...,0.00740556,0.0310319,99.3891,0.646001,99.542,99.6287,99.3837,98.9533,10.05,9.12935
3,130.5,-1.5,51.0043,34.2635,3.53263,4.36617,0.999434,0.998579,3.92281,8.40275,...,0.00189431,0.0154804,100,0.193622,100,100,100,100,0,0
4,14,40.5,613.947,190.339,4.26789,4.96893,0.990406,0.97013,4.08582,10.5326,...,0.00338907,0.0343177,97.7997,0.952054,99.2522,97.4626,97.7701,96.892,19.05,14.3018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
for j in seasons:
    print(j)
    dfy = dfx.query('season == "%s"' % j)

DJF
MAM
JJA
SON


In [56]:
dfy

,latitude,longitude,mwd,mwp,pp1d,swh,u10,v10,season,swh_nr_ad,swh_nr_rd,lfhs,lftm,wavePo
time,,,,,,,,,,,,,,
1979-09-01 00:00:00,55.0,-5.5,171.643600,3.948561,4.613307,0.798016,-0.359711,7.999001,SON,0.797772,0.0,0.012951,7.835113,0.563448
1979-09-01 01:00:00,55.0,-5.5,172.181961,3.936275,4.613307,0.747562,-0.434938,7.622383,SON,0.747347,0.0,0.011260,7.897246,0.479199
1979-09-01 02:00:00,55.0,-5.5,172.582993,3.919226,4.617116,0.698199,-0.274672,7.301781,SON,0.698013,0.0,0.009426,8.121426,0.404352
1979-09-01 03:00:00,55.0,-5.5,172.846680,3.902427,4.627158,0.651019,-0.223974,6.901989,SON,0.650859,0.0,0.007280,8.846772,0.339673
1979-09-01 04:00:00,55.0,-5.5,172.874146,3.875097,4.648628,0.610929,-0.711319,6.677952,SON,0.610797,0.0,0.004594,11.969558,0.289809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-30 19:00:00,55.0,-5.5,256.034363,5.903988,5.760216,1.718450,10.563018,4.914621,SON,1.664548,0.0,0.033071,11.199056,-2.709573
2018-11-30 20:00:00,55.0,-5.5,256.418884,5.865877,5.698230,1.645633,9.887604,4.756250,SON,1.595960,0.0,0.031844,10.998374,-2.465476
2018-11-30 21:00:00,55.0,-5.5,256.550720,5.840052,5.618930,1.561908,9.004498,4.667408,SON,1.516004,0.0,0.029468,10.552273,-2.176578


In [43]:
j

'SON'

In [48]:
dfx.query('season == "%s"' % j)

,latitude,longitude,mwd,mwp,pp1d,swh,u10,v10,season,wind,swh_nr_ad,swh_nr_rd,lfhs,lftm,wavePo
time,,,,,,,,,,,,,,,
1979-09-01 00:00:00,40.5,14.0,57.648521,3.076780,3.458433,0.649928,-6.817842,-3.863398,SON,7.836378,0.649928,0.000000,0.011597,6.710435,-0.338809
1979-09-01 01:00:00,40.5,14.0,59.972260,3.202144,3.614609,0.693291,-6.842373,-3.774555,SON,7.814431,0.693289,0.000000,0.012469,7.052588,-0.399767
1979-09-01 02:00:00,40.5,14.0,61.603821,3.296417,3.825152,0.719472,-6.978109,-3.737860,SON,7.916161,0.719468,0.000000,0.012661,6.845549,-0.439040
1979-09-01 03:00:00,40.5,14.0,62.620110,3.372137,3.907570,0.738017,-7.064785,-3.762966,SON,8.004442,0.738010,0.000000,0.013629,7.129169,-0.467873
1979-09-01 04:00:00,40.5,14.0,63.076073,3.429303,3.932503,0.751925,-6.992828,-3.844084,SON,7.979764,0.751915,0.000000,0.013629,7.129169,-0.490147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-30 19:00:00,40.5,14.0,199.885559,3.254295,6.100965,0.285027,-1.898607,3.823404,SON,4.268856,0.285026,0.285026,0.000503,42.781669,0.014124
2018-11-30 20:00:00,40.5,14.0,186.541901,3.141468,2.540074,0.311753,-2.408848,4.176844,SON,4.821677,0.311753,0.311753,0.005310,4.295016,-0.007166
2018-11-30 21:00:00,40.5,14.0,176.681122,3.051958,2.558428,0.346116,-2.547855,4.565046,SON,5.227926,0.346116,0.346116,0.006167,4.724435,-0.032059
